<a href="https://colab.research.google.com/github/kanpiromnida-cmd/madt7204-vibecoding/blob/main/RPG_Vibe_Coding_Quest_(Colab)_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
from IPython.display import HTML, display

# ฟังก์ชันสำหรับวาดหน้าปกด้วย CSS
def show_pixel_cover():
    display(HTML("""
    <style>
      /* นำเข้าฟอนต์ Pixel */
      @import url('https://fonts.googleapis.com/css2?family=Press+Start+2P&display=swap');

      /* พื้นหลังหลัก จำลองท้องฟ้าและพื้นหญ้า */
      .vibe-canvas {
        background: linear-gradient(to bottom, #5eb5f7 0%, #93c5fd 50%, #4ade80 50%, #166534 100%);
        padding: 40px 20px;
        text-align: center;
        font-family: 'Press Start 2P', cursive;
        border: 4px solid #000; /* ขอบดำสไตล์พิกเซล */
        position: relative;
        overflow: hidden;
      }

      /* ตัวป้ายสีน้ำตาล */
      .pixel-sign {
        background: #5c3333; /* สีน้ำตาลเข้ม */
        border: 4px solid #ffcc33; /* ขอบทอง */
        /* เพิ่มขอบในสีเข้มเพื่อให้ดูมีมิติ */
        box-shadow: inset 0 0 0 4px #3d2020;
        display: inline-block;
        padding: 25px 50px;
        position: relative;
        margin-bottom: 30px;
      }

      /* ตกแต่งมุมป้ายให้ดูเป็นพิกเซล (Pixel Corners) */
      .pixel-sign::before, .pixel-sign::after {
        content: ''; position: absolute; width: 10px; height: 10px; background: #ffcc33;
        border: 4px solid #3d2020; z-index: 2;
      }
      .pixel-sign::before { top: -8px; left: -8px; }
      .pixel-sign::after { top: -8px; right: -8px; }

      /* ข้อความหัวข้อ VIBE CODING */
      .main-title {
        font-size: 32px;
        color: #ffcc33; /* สีทอง/เหลือง */
        /* สร้างเงาแบบพิกเซล 3D */
        text-shadow: 4px 4px 0 #8b4513, 7px 7px 0 #000;
        margin: 0; line-height: 1.3;
      }

      /* ข้อความรอง */
      .sub-title {
        font-size: 12px; color: #fff; margin-top: 15px;
        text-shadow: 2px 2px 0 #000;
      }

      /* โซน Loading Bar */
      .loading-area { color: #fff; text-shadow: 2px 2px 0 #000; }
      .loading-text { font-size: 12px; margin-bottom: 10px; }

      /* กรอบหลอดโหลด */
      .progress-container {
        width: 350px; height: 25px; background: #2d3748;
        border: 4px solid #000; margin: 0 auto; padding: 3px;
      }
      /* ตัวหลอดสีเหลืองข้างใน */
      .progress-bar {
        width: 60%; /* เปอร์เซ็นต์การโหลด */
        height: 100%;
        background: linear-gradient(to bottom, #ffcc33, #f59e0b);
        border-right: 3px solid #000;
      }
    </style>

    <div class="vibe-canvas">
      <div class="pixel-sign">
        <h1 class="main-title">VIBE CODING</h1>
        <div class="sub-title">Email & Calendar Quest</div>
      </div>

      <div class="loading-area">
        <div class="loading-text">LOADING QUEST...</div>
        <div class="progress-container">
          <div class="progress-bar"></div>
        </div>
      </div>
    </div>
    """))

# เรียกใช้ฟังก์ชันเพื่อแสดงผล
show_pixel_cover()

# **Are you ready?**

In [ ]:
# @title
"""
RPG Vibe Coding: Google Auth + Calendar Quest (Colab-friendly)
"""

from __future__ import annotations

import re
import subprocess
import sys
import json
import warnings
import logging
import urllib.parse
import urllib.request
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Optional

try:
    from IPython.display import HTML, display
except Exception:  # pragma: no cover
    HTML = None

    def display(x):  # type: ignore
        print(x)


STATE_FILE = Path(".rpg_game_state.json")
QUOTA_PROJECT_ID: Optional[str] = None
logging.getLogger("googleapiclient.http").setLevel(logging.ERROR)


def load_stage() -> int:
    if not STATE_FILE.exists():
        return 1
    try:
        data = json.loads(STATE_FILE.read_text(encoding="utf-8"))
        if not bool(data.get("resume_allowed", False)):
            return 1
        stage = int(data.get("stage", 1))
        return max(1, min(stage, 8))
    except Exception:
        return 1


def save_stage(stage: int, resume_allowed: bool = False) -> None:
    STATE_FILE.write_text(
        json.dumps({"stage": stage, "resume_allowed": resume_allowed}),
        encoding="utf-8",
    )


def clear_stage() -> None:
    if STATE_FILE.exists():
        STATE_FILE.unlink()


def progress_from_stage(stage: int) -> int:
    return max(0, min(stage - 1, 7))


def in_colab() -> bool:
    return "google.colab" in sys.modules


def ui_bootstrap() -> None:
    if HTML is None:
        return
    display(
        HTML(
            """
            <style>
              @import url('https://fonts.googleapis.com/css2?family=Kanit:wght@400;600;700;800&family=Press+Start+2P&family=JetBrains+Mono:wght@500&display=swap');
              :root{
                --sky1:#8fd3ff; --sky2:#d9f2ff; --grass:#2fb44a; --dirt:#8b5a2b;
                --coin:#ffca28; --pipe:#22a03c; --brick:#cb6b36; --ok:#16a34a; --warn:#ea580c; --danger:#dc2626;
              }
              .rpg-wrap{font-family:'Kanit',system-ui; color:#0f172a;}
              .rpg-hero{
                position:relative;
                background:
                  radial-gradient(circle at 15% 25%, #ffffff 0 28px, transparent 29px),
                  radial-gradient(circle at 20% 25%, #ffffff 0 20px, transparent 21px),
                  radial-gradient(circle at 78% 22%, #ffffff 0 24px, transparent 25px),
                  radial-gradient(circle at 82% 22%, #ffffff 0 16px, transparent 17px),
                  linear-gradient(180deg,var(--sky2),var(--sky1));
                border:3px solid #3a6ea5; border-radius:20px; padding:18px 20px 28px 20px; max-width:980px; color:#102a43;
                overflow:hidden;
                box-shadow: 0 10px 0 #2f5a8b;
              }
              .rpg-hero:after{
                content:"";
                position:absolute; left:0; right:0; bottom:0; height:22px;
                background:
                  linear-gradient(180deg,var(--grass) 0 35%, #23883a 36% 100%);
                border-top:2px solid #1d6f2f;
              }
              .rpg-title{
                font-family:'Press Start 2P',monospace;
                font-size:24px; line-height:1.3; margin:0; color:#0b3b2e;
                text-shadow:2px 2px 0 #d7ffe6;
              }
              .rpg-sub{font-size:15px; color:#1f2937; margin-top:8px; font-weight:700;}
              .rpg-card{
                background:
                  linear-gradient(180deg,#fffdf6,#fff6e5);
                border:3px solid #4b5563; border-radius:14px; padding:14px 16px; max-width:980px; margin-top:14px; margin-bottom:12px;
                box-shadow:0 6px 0 #334155;
              }
              .rpg-stage{
                font-family:'Press Start 2P',monospace;
                font-size:18px; font-weight:800; color:#b45309; margin:0 0 10px 0;
              }
              .rpg-q{font-size:18px; font-weight:700; color:#0f172a; margin:0 0 8px 0;}
              .rpg-opt{font-family:'JetBrains Mono',monospace; font-size:15px; color:#1e293b; line-height:1.6;}
              .rpg-badge{
                display:inline-block; background:#1d4ed8; border:2px solid #1e3a8a; color:#eff6ff;
                font-weight:700; border-radius:999px; padding:4px 10px; margin-right:8px;
              }
              .rpg-ok{color:var(--ok); font-weight:700;}
              .rpg-warn{color:var(--warn); font-weight:700;}
              .rpg-err{color:var(--danger); font-weight:700;}
              .rpg-bar{
                height:16px; background:#0f172a; border-radius:999px; overflow:hidden; margin-top:10px;
                border:2px solid #111827;
              }
              .rpg-fill{
                height:100%;
                background: repeating-linear-gradient(
                  90deg,
                  #22c55e 0 18px,
                  #facc15 18px 36px,
                  #f97316 36px 54px
                );
              }
              .rpg-deco{font-size:20px; letter-spacing:2px; margin-bottom:8px;}
              .rpg-quest-title{
                font-family:'Press Start 2P',monospace;
                font-size:14px; color:#7c2d12; margin:0 0 8px 0;
              }
              .rpg-quest-text{font-size:15px; color:#1f2937; line-height:1.5;}
              .rpg-hp{font-family:'JetBrains Mono',monospace; font-size:14px; color:#7f1d1d; margin-top:6px;}
              .rpg-bossgate{
                background: linear-gradient(160deg,#0f172a 0%,#1e293b 55%,#0b1224 100%);
                border:3px solid #475569;
                box-shadow:0 8px 0 #111827;
                color:#e2e8f0;
              }
              .rpg-boss-title{
                font-family:'Press Start 2P',monospace;
                font-size:16px; line-height:1.4; color:#facc15; margin:0 0 8px 0;
                text-shadow:2px 2px 0 #7c2d12;
              }
              .rpg-boss-sub{font-size:14px; color:#bae6fd; font-weight:700; margin-bottom:10px;}
              .rpg-boss-grid{
                display:grid; grid-template-columns: 1.2fr 1fr; gap:10px; margin-top:8px;
              }
              .rpg-boss-panel{
                background:rgba(15,23,42,.55); border:1px solid #334155; border-radius:10px; padding:10px;
              }
              .rpg-boss-label{
                font-family:'Press Start 2P',monospace; font-size:11px; color:#fcd34d; margin:0 0 8px 0;
              }
              .rpg-boss-step{font-size:14px; color:#e2e8f0; line-height:1.55;}
              .rpg-boss-loot{
                display:inline-block; background:#7c3aed; border:1px solid #8b5cf6; color:#f3e8ff;
                border-radius:999px; font-weight:800; padding:4px 10px; margin:3px 4px 3px 0;
              }
              .rpg-boss-hp-wrap{display:flex; align-items:center; gap:8px; margin:8px 0 2px 0;}
              .rpg-boss-hp-text{
                font-family:'JetBrains Mono',monospace; font-size:12px; color:#fde68a; min-width:132px;
              }
              .rpg-boss-hp-bar{
                flex:1; height:12px; border-radius:999px; background:#1f2937; overflow:hidden; border:1px solid #334155;
              }
              .rpg-boss-hp-fill{
                height:100%; width:100%;
                background:repeating-linear-gradient(90deg,#ef4444 0 14px,#f97316 14px 28px,#facc15 28px 42px);
                animation:rpgPulse 1.6s linear infinite;
              }
              .rpg-boss-cmd{
                margin-top:10px; background:#020617; border:1px solid #334155; border-radius:10px; padding:10px;
              }
              .rpg-boss-cmd-title{
                font-family:'Press Start 2P',monospace; font-size:10px; color:#93c5fd; margin-bottom:6px;
              }
              .rpg-bossgate code{
                display:block; white-space:pre-wrap; word-break:break-word;
                background:#0b1224; color:#22d3ee; border:1px dashed #475569; padding:8px; border-radius:8px;
                font-size:13px;
              }
              @keyframes rpgPulse{
                0%{filter:saturate(1);}
                50%{filter:saturate(1.5);}
                100%{filter:saturate(1);}
              }
              @media (max-width:900px){
                .rpg-boss-grid{grid-template-columns:1fr;}
              }
            </style>
            """
        )
    )


def ui_hero(title: str, subtitle: str = "") -> None:
    if HTML is None:
        print(title)
        if subtitle:
            print(subtitle)
        return
    display(
        HTML(
            f"""
            <div class="rpg-wrap rpg-hero">
              <div class="rpg-deco">🪙 🧱 🍄 🟢</div>
              <div class="rpg-title">{title}</div>
              <div class="rpg-sub">{subtitle}</div>
            </div>
            """
        )
    )


def ui_card(text: str, kind: str = "info") -> None:
    if HTML is None:
        print(text)
        return
    cls = "rpg-ok" if kind == "ok" else "rpg-warn" if kind == "warn" else "rpg-err" if kind == "err" else ""
    display(
        HTML(
            f"""
            <div class="rpg-wrap rpg-card">
              <div class="{cls}">{text}</div>
            </div>
            """
        )
    )


def ui_stage(level: str, question: str, options: List[str], progress_done: int, progress_total: int) -> None:
    question = re.sub(r"^\s*RO-B0T:\s*", "Question: ", question, flags=re.IGNORECASE)
    pct = int((progress_done / progress_total) * 100)
    if HTML is None:
        print(f"\n{level}\n{question}")
        for op in options:
            print(op)
        print(f"Progress: {pct}%")
        return
    options_html = "<br>".join(options)
    display(
        HTML(
            f"""
            <div class="rpg-wrap rpg-card">
              <div class="rpg-deco">🟩 🧱 🪙</div>
              <div class="rpg-stage">{level}</div>
              <div class="rpg-q">{question}</div>
              <div class="rpg-opt">{options_html}</div>
              <div style="margin-top:10px;">
                <span class="rpg-badge">Progress {progress_done}/{progress_total}</span>
                <span class="rpg-badge">{pct}%</span>
              </div>
              <div class="rpg-bar"><div class="rpg-fill" style="width:{pct}%"></div></div>
            </div>
            """
        )
    )


def ui_project_key_quest(attempt: int, suggested_ids: List[str]) -> None:
    monster_cycle = [
        "Quota Slime",
        "Scope Goblin",
        "403 Dragon",
        "API Wyvern",
    ]
    monster = monster_cycle[(attempt - 1) % len(monster_cycle)]
    hp_max = 5
    hp_now = max(1, hp_max - (attempt - 1))
    hp_bar = ("🟥" * hp_now) + ("⬛" * (hp_max - hp_now))

    if HTML is None:
        print(f"\n[Side Quest] Hunt {monster}")
        if suggested_ids:
            print("Objective: เลือก Project Key จาก Loot Scanner ได้เลย (กด Enter = เลือก #1)")
        else:
            print("Objective: ใส่ Project Key (Project ID) ที่มีอยู่แล้วได้เลย")
        if suggested_ids:
            print("Loot candidates:")
            for idx, pid in enumerate(suggested_ids, start=1):
                print(f"{idx}) {pid}")
            print("Hint: กด Enter เพื่อเลือก #1 อัตโนมัติ")
            print("Optional map (ถ้าไม่มี key ที่ต้องการ): https://console.cloud.google.com/cloud-resource-manager")
        print(f"Boss HP: {hp_bar}")
        return

    loot_html = ""
    objective_html = (
        "🎯 Objective: ใส่ <b>Project Key (Project ID)</b> ที่มีอยู่แล้วได้เลย<br>"
        "ไม่จำเป็นต้องเปิด Cloud Resource Manager ทุกครั้ง"
    )
    map_html = (
        "🔗 Optional Map: <a href=\"https://console.cloud.google.com/cloud-resource-manager\" target=\"_blank\">"
        "https://console.cloud.google.com/cloud-resource-manager</a><br>"
        "ใช้ค่าคอลัมน์ <b>Project ID</b> เท่านั้น (ไม่ใช่ Name / Number)"
    )
    if suggested_ids:
        rows = "<br>".join(f"{idx + 1}) <code>{pid}</code>" for idx, pid in enumerate(suggested_ids))
        objective_html = (
            "🎯 Objective: <b>เลือก Project Key จาก Loot Scanner ด้านล่างได้เลย</b> "
            "(ไม่ต้องออกไปหาเพิ่ม)"
        )
        map_html = (
            "🧭 Optional: ถ้าอยากหาเพิ่มค่อยใช้ Map นี้:<br>"
            "<a href=\"https://console.cloud.google.com/cloud-resource-manager\" target=\"_blank\">"
            "https://console.cloud.google.com/cloud-resource-manager</a>"
        )
        loot_html = (
            "<div class='rpg-quest-text'><b>🎒 Loot Scanner เจอ Project Keys:</b><br>"
            f"{rows}<br>พิมพ์ได้ทั้ง <b>เลขลำดับ</b> หรือ <b>Project ID ตรงๆ</b><br>"
            "✨ <b>Recommended:</b> กด Enter เพื่อเลือก <b>#1</b> อัตโนมัติ</div>"
        )

    display(
        HTML(
            f"""
            <div class="rpg-wrap rpg-card">
              <div class="rpg-quest-title">🗺️ Side Quest: Hunt the Project Key</div>
              <div class="rpg-quest-text">
                👾 Monster: <b>{monster}</b><br>
                {objective_html}
              </div>
              <div class="rpg-hp">Boss HP: {hp_bar} · Attempt {attempt}</div>
              <div class="rpg-quest-text" style="margin-top:8px;">
                {map_html}
              </div>
              {loot_html}
            </div>
            """
        )
    )


def ui_auth_key_boss_gate(cmd: str) -> None:
    if HTML is None:
        print("\n[Boss Gate] Hunt the Auth Key")
        print("1) ถามต่อให้พิมพ์ Y")
        print("2) เปิดลิงก์และ login ให้ครบ")
        print("3) เอา verification code มาใส่กลับ")
        print(f"Run: {cmd}")
        return

    display(
        HTML(
            f"""
            <div class="rpg-wrap rpg-card rpg-bossgate">
              <div class="rpg-boss-title">⚔️ BOSS GATE: The GCE Sentinel</div>
              <div class="rpg-boss-sub">Quest Objective: ล่า <b>Auth Key</b> เพื่อทำลายโล่แล้วเปิดทางไปด่านถัดไป</div>

              <div class="rpg-boss-hp-wrap">
                <div class="rpg-boss-hp-text">Sentinel Shield 100%</div>
                <div class="rpg-boss-hp-bar"><div class="rpg-boss-hp-fill"></div></div>
              </div>

              <div class="rpg-boss-grid">
                <div class="rpg-boss-panel">
                  <div class="rpg-boss-label">MISSION STEPS</div>
                  <div class="rpg-boss-step">
                    1) เจอคำถาม <b>Do you want to continue</b> ให้พิมพ์ <b>Y</b><br>
                    2) เปิดลิงก์ที่ระบบให้ แล้ว Login ใน browser ให้จบ<br>
                    3) นำ <b>verification code</b> กลับมาใส่ใน output นี้
                  </div>
                </div>
                <div class="rpg-boss-panel">
                  <div class="rpg-boss-label">LOOT / REWARD</div>
                  <div class="rpg-boss-step">
                    <span class="rpg-boss-loot">Auth Key</span>
                    <span class="rpg-boss-loot">Calendar Scope</span>
                    <span class="rpg-boss-loot">Gate Pass</span><br>
                    เสร็จแล้วระบบจะ verify อัตโนมัติทันที
                  </div>
                </div>
              </div>

              <div class="rpg-boss-cmd">
                <div class="rpg-boss-cmd-title">CAST SPELL IN THIS CELL</div>
                <code>!{cmd}</code>
              </div>
            </div>
            """
        )
    )


def ask_until_correct(
    level: str, question: str, options: List[str], correct: str, progress_done: int, progress_total: int
) -> str:
    ui_stage(level, question, options, progress_done, progress_total)
    while True:
        ans = input("\nYour answer (a/b): ").strip().upper()
        if ans == correct:
            ui_card("✅ Correct!", "ok")
            return ans
        ui_card("❌ ยังไม่ถูก ลองใหม่อีกครั้ง", "err")


def run_cmd(cmd: List[str], fail_message: str) -> bool:
    printable = " ".join(cmd)
    print(f"[RUN] {printable}", flush=True)
    try:
        subprocess.run(cmd, check=True, timeout=90)
        return True
    except subprocess.TimeoutExpired:
        ui_card(f"{fail_message}<br>Reason: command timed out (90s): <code>{printable}</code>", "err")
        return False
    except Exception as exc:
        ui_card(f"{fail_message}<br>Reason: {exc}", "err")
        return False


def run_ipython_cmd(cmd: str, fail_message: str) -> bool:
    """
    Run shell command via IPython system so Colab can stay interactive for auth/code paste.
    """
    try:
        from IPython import get_ipython

        shell = get_ipython()
        if shell is None:
            raise RuntimeError("IPython shell not available")
        tag = "[RUN]"
        if "gcloud auth application-default login" in cmd:
            tag = "[QUEST-RUN]"
        print(f"{tag} {cmd}", flush=True)
        rc = shell.system(cmd)
        if rc in (0, None):
            return True
        ui_card(f"{fail_message}<br>Reason: exit status {rc}", "err")
        return False
    except Exception as exc:
        ui_card(f"{fail_message}<br>Reason: {exc}", "err")
        return False


def ensure_deps() -> None:
    try:
        import googleapiclient  # type: ignore
        import google.auth  # type: ignore
        import ipywidgets  # type: ignore
        import pytz  # type: ignore
        import pydata_google_auth  # type: ignore

        ui_card("✅ Dependencies already available", "ok")
        return
    except Exception:
        pass

    ui_card("Installing dependencies... (this may take up to 1-2 minutes)", "warn")
    run_cmd(
        [
            sys.executable,
            "-m",
            "pip",
            "install",
            "--upgrade",
            "google-api-python-client",
            "google-auth-httplib2",
            "google-auth-oauthlib",
            "ipywidgets",
            "pytz",
            "pydata-google-auth",
        ],
        "⚠️ Install failed. Please run pip install manually in Colab.",
    )


def colab_auth(force_popup: bool = False) -> None:
    try:
        from google.colab import auth

        if force_popup:
            # Reset local auth cache so Colab asks consent again on this gate.
            try:
                subprocess.run(["gcloud", "auth", "application-default", "revoke", "--quiet"], check=False)
                subprocess.run(["gcloud", "auth", "revoke", "--all", "--quiet"], check=False)
            except Exception:
                pass
            try:
                adc = Path.home() / ".config" / "gcloud" / "application_default_credentials.json"
                if adc.exists():
                    adc.unlink()
            except Exception:
                pass
            ui_card("🪪 กำลังบังคับเปิด OAuth popup ใหม่ที่ด่านนี้...", "warn")
            auth.authenticate_user(clear_output=False)
        else:
            auth.authenticate_user()
        ui_card("✅ Authenticated in Colab", "ok")
    except Exception as exc:
        ui_card(f"⚠️ Colab auth failed: {exc}", "warn")


def connect_calendar(quota_project_id: Optional[str] = None):
    import google.auth
    from google.auth.transport.requests import Request
    from googleapiclient.discovery import build

    scopes = ["https://www.googleapis.com/auth/calendar"]
    # Hide noisy SDK warning; quota project is handled in-game via QUOTA_PROJECT_ID.
    with warnings.catch_warnings():
        warnings.filterwarnings(
            "ignore",
            message="Your application has authenticated using end user credentials from Google Cloud SDK without a quota project.*",
            category=UserWarning,
        )
        creds, _ = google.auth.default(scopes=scopes)
    if quota_project_id and hasattr(creds, "with_quota_project"):
        creds = creds.with_quota_project(quota_project_id)
    # Refresh token early to fail fast when auth is not ready.
    creds.refresh(Request())
    service = build("calendar", "v3", credentials=creds, cache_discovery=False)
    ui_card("✅ Connected to Google Calendar API", "ok")
    return service


def connect_calendar_loose(quota_project_id: Optional[str] = None):
    """
    Build Calendar service without strict token refresh/scope checks.
    Used in early levels so gate can enforce gcloud login at the intended step.
    """
    import google.auth
    from googleapiclient.discovery import build

    with warnings.catch_warnings():
        warnings.filterwarnings(
            "ignore",
            message="Your application has authenticated using end user credentials from Google Cloud SDK without a quota project.*",
            category=UserWarning,
        )
        creds, _ = google.auth.default()
    if quota_project_id and hasattr(creds, "with_quota_project"):
        creds = creds.with_quota_project(quota_project_id)
    service = build("calendar", "v3", credentials=creds, cache_discovery=False)
    ui_card("✅ Connected to Google Calendar API", "ok")
    return service


def connect_calendar_with_scope_reauth(quota_project_id: Optional[str] = None):
    """
    Connect and validate calendar scope.
    Do not perform repeated re-auth loops here; gate will handle forced gcloud login.
    """
    try:
        service = connect_calendar(quota_project_id)
    except Exception as exc:
        ui_card(f"⚠️ เชื่อม Calendar ครั้งแรกไม่สำเร็จ: {exc}", "err")
        service = None
    if service is None:
        return None
    ok, _ = has_calendar_scope(service)
    if ok:
        return service
    ui_card(
        "⚠️ ยังไม่มี Calendar scope จาก ADC ปัจจุบัน<br>"
        "กำลังรัน gcloud login แบบบังคับใน gate ถัดไป",
        "warn",
    )
    return None


def has_calendar_scope(service) -> tuple[bool, str]:
    try:
        # Lightweight call to confirm token actually has calendar scope.
        service.calendarList().list(maxResults=1).execute()
        return True, "ok"
    except Exception as exc:
        msg = str(exc)
        low = msg.lower()
        if "insufficientpermissions" in low or "insufficient authentication scopes" in low:
            return False, (
                "บัญชียังไม่มี Calendar scope. กรุณา re-auth ด้วยคำสั่ง:<br>"
                "<code>!gcloud auth application-default login "
                "--scopes=https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/calendar</code>"
            )
        if "requires a quota project" in low or "without a quota project" in low:
            return False, (
                "ADC ยังไม่ได้ผูก quota project สำหรับ Calendar API<br>"
                "ไปด่านถัดไปเพื่อตั้งค่า quota project ก่อนใช้งาน Calendar"
            )
        return False, f"ตรวจ scope ไม่ผ่าน: {msg}"


def verify_calendar_scope_without_calendar_api(quota_project_id: Optional[str] = None) -> tuple[bool, str]:
    """
    Verify that ADC has calendar scope by checking token scopes (tokeninfo),
    so this step does not require Calendar API/quota-project yet.
    """
    import google.auth
    from google.auth.transport.requests import Request

    scopes = [
        "https://www.googleapis.com/auth/cloud-platform",
        "https://www.googleapis.com/auth/calendar",
    ]
    with warnings.catch_warnings():
        warnings.filterwarnings(
            "ignore",
            message="Your application has authenticated using end user credentials from Google Cloud SDK without a quota project.*",
            category=UserWarning,
        )
        creds, _ = google.auth.default(scopes=scopes)
    if quota_project_id and hasattr(creds, "with_quota_project"):
        creds = creds.with_quota_project(quota_project_id)

    creds.refresh(Request())
    calendar_scope = "https://www.googleapis.com/auth/calendar"

    granted = set(getattr(creds, "granted_scopes", []) or [])
    if calendar_scope in granted:
        return True, "ok"

    token = getattr(creds, "token", None)
    if not token:
        return False, "ไม่พบ ADC token หลัง login. ให้รัน gcloud auth application-default login ใหม่"

    try:
        url = "https://oauth2.googleapis.com/tokeninfo?access_token=" + urllib.parse.quote(token, safe="")
        with urllib.request.urlopen(url, timeout=20) as resp:
            payload = json.loads(resp.read().decode("utf-8"))
        token_scopes = set((payload.get("scope", "") or "").split())
        if calendar_scope in token_scopes:
            return True, "ok"
        return False, (
            "🗝️ ยังไม่ได้ <b>Auth Key (Calendar scope)</b> จาก ADC token<br>"
            "ให้กลับไปล่า Key ใหม่ด้วยคำสั่ง:<br>"
            "<code>!gcloud auth application-default login "
            "--scopes=https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/calendar</code>"
        )
    except Exception as exc:
        return False, (
            "ไม่สามารถตรวจ Calendar scope จาก tokeninfo อัตโนมัติได้<br>"
            f"Reason: {exc}<br>"
            "ให้ re-auth แล้ว Run เกมใหม่อีกครั้ง"
        )


def verify_adc_ready() -> bool:
    ok = run_cmd(
        ["gcloud", "auth", "application-default", "print-access-token", "--quiet"],
        "⚠️ ยังยืนยัน ADC token ไม่ได้",
    )
    if not ok:
        return False
    scope_ok, scope_msg = verify_calendar_scope_without_calendar_api(QUOTA_PROJECT_ID)
    if not scope_ok:
        ui_card(f"⚠️ {scope_msg}", "err")
        return False
    ui_card("✅ ได้ Auth Key แล้ว (ADC token + Calendar scope verified)", "ok")
    return True


def gcloud_login_gate() -> bool:
    """
    Single-flow gate for notebook UX (no switching cells).
    """
    ui_card(
        "🚪 Quest Gate: ต้องออกล่าหา <b>Auth Key</b> ก่อน ถึงจะเปิดทางไปด่านถัดไปได้",
        "warn",
    )
    cmd_no_bang = (
        "gcloud auth application-default login "
        "--scopes=https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/calendar"
    )
    cmd = f"!{cmd_no_bang}"

    # If token+scope already ready, continue without forcing login again.
    if verify_adc_ready():
        ui_card("✅ มี Auth Key ติดตัวอยู่แล้ว ข้ามการล่าซ้ำ", "ok")
        return True

    ui_auth_key_boss_gate(cmd_no_bang)

    ok = run_ipython_cmd(
        cmd_no_bang,
        "⚠️ ภารกิจล่า Auth Key ล้มเหลว (อาจยกเลิกกลางทาง หรือยัง Login ไม่ครบ)",
    )
    if not ok:
        return False

    if verify_adc_ready():
        ui_card("🏆 เคลียร์ด่าน Gate สำเร็จ! ได้ Auth Key แล้ว ไปด่านถัดไปได้", "ok")
        return True

    ui_card(
        "⛺ ยังผ่าน Gate ไม่ได้ (ยัง verify Auth Key ไม่ผ่าน)<br>"
        "ให้ทำภารกิจตามข้อความด้านบนให้ครบ แล้ว Run เกมซ้ำอีกครั้ง",
        "err",
    )
    return False


def list_candidate_projects(limit: int = 10) -> List[str]:
    """
    Fetch project IDs to help users copy/paste quickly.
    """
    try:
        res = subprocess.run(
            ["gcloud", "projects", "list", "--format=value(projectId)", f"--limit={limit}"],
            check=True,
            capture_output=True,
            text=True,
            timeout=30,
        )
    except Exception:
        return []

    ids = [line.strip() for line in (res.stdout or "").splitlines() if line.strip()]
    return ids


def set_quota_project_gate() -> None:
    # Common copy/paste issue: Unicode dashes in project id (e.g. – — −)
    dash_variants = "\u2010\u2011\u2012\u2013\u2014\u2015\u2212"
    project_re = re.compile(r"^[a-z][a-z0-9-]{4,28}[a-z0-9]$")

    suggested_ids = list_candidate_projects(limit=10)
    attempt = 1
    while True:
        ui_project_key_quest(attempt, suggested_ids)
        if suggested_ids:
            raw = input("\n🗝️ Enter Project Key (เลขลำดับ/Project ID, Enter=#1): ").strip()
            if raw == "":
                raw = "1"
                ui_card("⚡ Auto pick: เลือก #1 (Recommended)", "ok")
        else:
            raw = input("\n🗝️ Enter Project Key (GCP Project ID หรือเลขลำดับ): ").strip()
        if raw.isdigit() and suggested_ids:
            idx = int(raw) - 1
            if 0 <= idx < len(suggested_ids):
                raw = suggested_ids[idx]
                ui_card(f"🎒 Loot acquired: <code>{raw}</code>", "ok")
        project_id = raw.translate(str.maketrans({ch: "-" for ch in dash_variants})).strip().lower()

        if not project_id:
            ui_card("❌ ยังไม่ได้ Project Key ลองล่ามอนอีกรอบ", "err")
            attempt += 1
            continue

        if project_id != raw:
            ui_card(f"ℹ️ Normalized Project ID: <code>{project_id}</code>", "warn")

        if not project_re.fullmatch(project_id):
            ui_card(
                "⚔️ Project Key ปลอม! รูปแบบไม่ถูกต้อง (ต้องเป็น a-z, 0-9, - และขึ้นต้นด้วยตัวอักษร)",
                "err",
            )
            attempt += 1
            continue

        global QUOTA_PROJECT_ID
        ok = run_cmd(
            ["gcloud", "auth", "application-default", "set-quota-project", project_id],
            "⚠️ ตั้ง quota project ผ่าน gcloud ไม่สำเร็จ",
        )
        if not ok:
            ui_card("👾 มอนสเตอร์กันทางไว้: set-quota-project ยังไม่ผ่าน", "err")
            action = input("\nพิมพ์ R=ลองใหม่ หรือ Q=ออกเกม: ").strip().upper()
            if action == "Q":
                raise RuntimeError("Game stopped: quota project setup is required to continue.")
            attempt += 1
            continue

        QUOTA_PROJECT_ID = project_id
        try:
            service = connect_calendar_with_scope_reauth(QUOTA_PROJECT_ID)
            if service is None:
                raise RuntimeError("ยังเชื่อม Calendar ไม่สำเร็จหลังตั้ง quota project")
            scope_ok, scope_msg = has_calendar_scope(service)
            if not scope_ok:
                raise RuntimeError(scope_msg)
            ui_card(f"🏆 Boss defeated! Project Key activated: <code>{project_id}</code>", "ok")
            return
        except Exception as exc:
            msg = str(exc)
            QUOTA_PROJECT_ID = None

        hint = (
            "ตรวจว่า Project ID ถูกต้องและบัญชีนี้มีสิทธิ์ "
            "<code>serviceusage.services.use</code> บนโปรเจกต์นั้น"
        )
        low = msg.lower()
        if "permission" in low or "serviceusage.services.use" in low or "quota_project" in low:
            hint = (
                "บัญชีนี้ยังไม่มีสิทธิ์ใช้ quota project นี้ "
                "(ต้องมีสิทธิ์ serviceusage.services.use)"
            )
        elif "not found" in low or "does not exist" in low:
            hint = "ไม่พบ Project ID นี้ใน GCP (อาจพิมพ์ผิด)"
        elif "api has not been used" in low or "accessnotconfigured" in low or "not enabled" in low:
            hint = (
                "ยังไม่ได้เปิด Google Calendar API ในโปรเจกต์นี้ "
                "ให้เปิดที่ Google Cloud Console > APIs & Services > Enabled APIs"
            )

        ui_card(
            "👹 บอสตีกลับ! ยังตั้ง quota project ไม่ผ่าน<br>"
            f"Reason: {msg}<br>Hint: {hint}<br>"
            "กลับไปล่า Project Key ใหม่ แล้วลองอีกครั้ง",
            "err",
        )
        action = input("\nพิมพ์ R=ลองใหม่ หรือ Q=ออกเกม: ").strip().upper()
        if action == "Q":
            raise RuntimeError("Game stopped: quota project setup is required to continue.")
        attempt += 1


def fmt_dt(x: Optional[str], tz):
    if not x:
        return "-"
    if "T" in x:
        dt = datetime.fromisoformat(x.replace("Z", "+00:00")).astimezone(tz)
        return dt.strftime("%Y-%m-%d %H:%M")
    return f"{x} (all-day)"


def list_events_today_and_range(service, tz_name: str = "Asia/Bangkok") -> None:
    import pytz
    from html import escape

    tz = pytz.timezone(tz_name)
    today_local = datetime.now(tz).date()
    first_day = today_local.replace(day=1)
    next_month = (first_day + timedelta(days=32)).replace(day=1)
    last_day = next_month - timedelta(days=1)
    list_start = first_day.isoformat()
    list_end = last_day.isoformat()

    def range_inclusive(start_ymd: str, end_ymd: str):
        sd = tz.localize(datetime.strptime(start_ymd, "%Y-%m-%d"))
        ed = tz.localize(datetime.strptime(end_ymd, "%Y-%m-%d")) + timedelta(days=1)
        return sd, ed

    def fetch(sd_dt, ed_dt, max_results=50):
        items = []
        token = None
        while True:
            try:
                res = service.events().list(
                    calendarId="primary",
                    timeMin=sd_dt.isoformat(),
                    timeMax=ed_dt.isoformat(),
                    singleEvents=True,
                    orderBy="startTime",
                    maxResults=min(250, max_results),
                    pageToken=token,
                ).execute()
            except Exception as exc:
                msg = str(exc)
                low = msg.lower()
                if "insufficientpermissions" in low or "insufficient authentication scopes" in low:
                    ui_card(
                        "⚠️ Calendar scope ยังไม่พอสำหรับ list events<br>"
                        "ให้รันคำสั่งนี้แล้วเริ่มเกมใหม่:<br>"
                        "<code>!gcloud auth application-default login "
                        "--scopes=https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/calendar</code>",
                        "err",
                    )
                    return []
                raise
            items.extend(res.get("items", []))
            token = res.get("nextPageToken")
            if not token or len(items) >= max_results:
                break
        return items[:max_results]

    def dedupe(items):
        out = []
        seen = set()
        for ev in items:
            summary = (ev.get("summary") or "").strip()
            st = ev.get("start", {}).get("dateTime") or ev.get("start", {}).get("date") or ""
            en = ev.get("end", {}).get("dateTime") or ev.get("end", {}).get("date") or ""
            key = (summary, st, en)
            if key in seen:
                continue
            seen.add(key)
            out.append(ev)
        return out

    def render_items(title, subtitle, items):
        if HTML is None:
            print(f"\n=== {title} ({len(items)} events) ===")
            if not items:
                print("No events found.")
                return
            for i, ev in enumerate(items, start=1):
                st = ev.get("start", {}).get("dateTime") or ev.get("start", {}).get("date")
                en = ev.get("end", {}).get("dateTime") or ev.get("end", {}).get("date")
                print(f"{i:02d}. {fmt_dt(st, tz)} -> {fmt_dt(en, tz)} | {ev.get('summary', '(no title)')}")
            return

        if not items:
            ui_card("📭 Mission Board ว่างอยู่ตอนนี้ (ไม่พบอีเวนต์ในช่วงนี้)", "warn")
            return

        rows = ""
        for i, ev in enumerate(items, start=1):
            st = ev.get("start", {}).get("dateTime") or ev.get("start", {}).get("date")
            en = ev.get("end", {}).get("dateTime") or ev.get("end", {}).get("date")
            summary = escape(ev.get("summary", "(no title)"))
            st_fmt = fmt_dt(st, tz)
            en_fmt = fmt_dt(en, tz)
            if "T" in (st or ""):
                date_text = st_fmt[:10]
                time_text = f"{st_fmt[11:16]}–{en_fmt[11:16]}"
            else:
                date_text = st_fmt.split(" ")[0]
                time_text = "All-day"
            rows += f"""
            <tr>
              <td style='padding:10px 8px;border-bottom:1px solid #e2e8f0;font-weight:800;color:#1e3a8a;width:56px;'>#{i:02d}</td>
              <td style='padding:10px 8px;border-bottom:1px solid #e2e8f0;width:140px;'>{date_text}</td>
              <td style='padding:10px 8px;border-bottom:1px solid #e2e8f0;width:120px;'>{time_text}</td>
              <td style='padding:10px 8px;border-bottom:1px solid #e2e8f0;font-weight:700;color:#0f172a;'>{summary}</td>
            </tr>
            """

        display(
            HTML(
                f"""
                <div class="rpg-wrap rpg-card">
                  <div class="rpg-stage">📅 Mission Board: Event List</div>
                  <div class="rpg-q" style="font-size:15px;color:#475569;">{escape(subtitle)} · ทั้งหมด {len(items)} events</div>
                  <table style="width:100%;border-collapse:collapse;margin-top:8px;background:#fff;">
                    <thead>
                      <tr style="background:#eff6ff;text-align:left;color:#1e3a8a;">
                        <th style="padding:8px;">ID</th>
                        <th style="padding:8px;">Date</th>
                        <th style="padding:8px;">Time</th>
                        <th style="padding:8px;">Title</th>
                      </tr>
                    </thead>
                    <tbody>{rows}</tbody>
                  </table>
                </div>
                """
            )
        )

    sd, ed = range_inclusive(list_start, list_end)
    items = fetch(sd, ed, max_results=50)
    items = dedupe(items)
    render_items("LIST EVENTS", f"{list_start} → {list_end}", items)


def quick_add_and_notify(service, tz_name: str = "Asia/Bangkok") -> None:
    import pytz

    tz = pytz.timezone(tz_name)
    email_re = re.compile(r"[^@\s]+@[^@\s]+\.[^@\s]+")

    def read_or_quit(prompt: str, default: Optional[str] = None) -> Optional[str]:
        raw = input(prompt).strip()
        if raw.lower() == "q":
            ui_card("⏸️ Pause meeting quest แล้ว (พิมพ์ q) ไว้ค่อยมานัดต่อได้", "warn")
            return None
        if raw == "" and default is not None:
            return default
        return raw

    def parse_attendees(raw: str):
        raw = (raw or "").strip()
        if not raw:
            return []
        parts = [p.strip() for p in raw.split(",") if p.strip()]
        bad = [p for p in parts if not email_re.fullmatch(p)]
        if bad:
            raise ValueError("Invalid email(s): " + ", ".join(bad))
        seen = set()
        out = []
        for p in parts:
            k = p.lower()
            if k not in seen:
                seen.add(k)
                out.append({"email": p})
        return out

    def exists_duplicate(start_dt, end_dt, title_str):
        res = service.events().list(
            calendarId="primary",
            timeMin=(start_dt - timedelta(minutes=1)).isoformat(),
            timeMax=(end_dt + timedelta(minutes=1)).isoformat(),
            singleEvents=True,
            orderBy="startTime",
            maxResults=50,
        ).execute()
        for ev in res.get("items", []):
            if (
                (ev.get("summary", "").strip() == title_str.strip())
                and ev.get("start", {}).get("dateTime") == start_dt.isoformat()
                and ev.get("end", {}).get("dateTime") == end_dt.isoformat()
            ):
                return True
        return False

    ui_card("📅 นัด Meeting มาได้เลย", "warn")
    print("Guardrail: create only 08:00–18:00 (15-min steps)")
    print("Tip: พิมพ์ q ช่องไหนก็ได้ เพื่อหยุดแล้วค่อยมานัดต่อทีหลัง")

    title = read_or_quit("\nTitle: ")
    if title is None:
        return
    date = read_or_quit("Date (YYYY-MM-DD) [2026-02-20]: ", default="2026-02-20")
    if date is None:
        return
    start_time = read_or_quit("Start time HH:MM [09:00]: ", default="09:00")
    if start_time is None:
        return
    end_time = read_or_quit("End time HH:MM [10:00]: ", default="10:00")
    if end_time is None:
        return
    attendees = read_or_quit("Attendees emails (comma-separated): ", default="")
    if attendees is None:
        return
    agenda = read_or_quit("agenda_or_description: ", default="")
    if agenda is None:
        return
    send_invites_raw = read_or_quit("Send invites? (y/n) [y]: ", default="y")
    if send_invites_raw is None:
        return
    send_invites = send_invites_raw.lower() == "y"
    confirm_raw = read_or_quit("Type RUN to create event (or q to pause): ", default="")
    if confirm_raw is None:
        return
    confirm_create = confirm_raw.upper() == "RUN"

    if not confirm_create:
        ui_card("✅ กรอกข้อมูลเสร็จแล้วกด Run เพื่อส่งเมล์", "ok")
        return

    if not title:
        ui_card("❌ Title is required.", "err")
        return

    time_pattern = re.compile(r"^\d{2}:\d{2}$")
    if not time_pattern.fullmatch(start_time) or not time_pattern.fullmatch(end_time):
        ui_card("❌ Time format must be HH:MM", "err")
        return
    if not ("08:00" <= start_time <= "18:00" and "08:00" <= end_time <= "18:00"):
        ui_card("❌ Time must be within 08:00–18:00", "err")
        return
    if end_time <= start_time:
        ui_card("❌ End time must be after start time.", "err")
        return

    start_dt = tz.localize(datetime.strptime(f"{date} {start_time}", "%Y-%m-%d %H:%M"))
    end_dt = tz.localize(datetime.strptime(f"{date} {end_time}", "%Y-%m-%d %H:%M"))
    if exists_duplicate(start_dt, end_dt, title):
        ui_card("⚠️ Duplicate found, event NOT created.", "warn")
        return

    try:
        attendees_list = parse_attendees(attendees)
    except Exception as exc:
        ui_card(f"❌ Attendees error: {exc}", "err")
        return

    body = {
        "summary": title,
        "start": {"dateTime": start_dt.isoformat(), "timeZone": tz_name},
        "end": {"dateTime": end_dt.isoformat(), "timeZone": tz_name},
    }
    if agenda:
        body["description"] = agenda
    if attendees_list:
        body["attendees"] = attendees_list

    created = service.events().insert(
        calendarId="primary",
        body=body,
        sendUpdates="all" if (send_invites and attendees_list) else "none",
    ).execute()
    ui_card("✅ Event created successfully", "ok")
    print(f"Open in Google Calendar: {created.get('htmlLink', '-')}")


def main() -> None:
    ui_bootstrap()
    stage = load_stage()
    if stage > 1:
        ui_card(f"ℹ️ Resume from gcloud gate checkpoint Lv.{stage} (auto)", "info")
    ui_hero("Welcome to My Word!!!", f"RPG Vibe Coding for Email & Calendar Quest · checkpoint Lv.{stage}")
    service = None
    total_steps = 7
    done = progress_from_stage(stage)

    if stage <= 1:
        ask_until_correct(
            "Lv.1 Authentication Gate",
            "RO-B0T: จะให้ระบบส่งเมลแทนได้ ต้องทำอะไรก่อน?",
            [
                "A) พิมพ์รหัสผ่านลงในโน้ต แล้วฝาก RO-B0T ถือไว้",
                "B) ทำ OAuth ขอ permission แบบถูกต้อง",
            ],
            "B",
            done,
            total_steps,
        )
        ensure_deps()
        if in_colab():
            colab_auth()
        stage = 2
        save_stage(stage)
        done = progress_from_stage(stage)

    if stage <= 2:
        ask_until_correct(
            "Lv.2 Target Setup",
            "RO-B0T: ส่วนไหน 'ต้องมี' ก่อนประกอบเมลให้ส่งได้?",
            [
                "A) To + Subject อย่างน้อยต้องครบ",
                "B) เขียนเนื้อหาอลังการก่อน เดี๋ยวระบบเดาว่าจะส่งหาใครเอง",
            ],
            "A",
            done,
            total_steps,
        )
        try:
            service = connect_calendar_loose(QUOTA_PROJECT_ID)
        except Exception as exc:
            ui_card(
                "⛔ เชื่อม Calendar เบื้องต้นไม่สำเร็จใน Lv.2<br>"
                f"Reason: {exc}",
                "err",
            )
            save_stage(2)
            return
        stage = 3
        save_stage(stage)
        done = progress_from_stage(stage)

    if stage <= 3:
        ask_until_correct(
            "Obstacle #1 Spam Trap",
            "RO-B0T: หัวข้อเมลแบบไหน 'ปลอดภัย' และ 'ค้นหาเจอง่าย' สุด?",
            [
                "A) ด่วนมาก!!!!! งานจะพังแล้ว!!!!",
                "B) [ชื่อวิชา] ขอคำแนะนำเรื่อง (หัวข้อ) — ชื่อ-นามสกุล/รหัส",
            ],
            "B",
            done,
            total_steps,
        )
        if not gcloud_login_gate():
            save_stage(3, resume_allowed=True)
            return
        stage = 4
        save_stage(stage)
        done = progress_from_stage(stage)

    if stage <= 4:
        ask_until_correct(
            "Lv.3 Etiquette Check",
            "RO-B0T: เปิดเมลแบบไหนสุภาพ แต่ไม่เยิ่นเย้อ?",
            [
                "A) สวัสดีครับอาจารย์!!! ผมขอรบกวนอาจารย์มากๆๆๆๆ",
                "B) เรียนอาจารย์... ผม/หนู (ชื่อ) จากวิชา... ขอรบกวนสอบถามเรื่อง...สั้นๆ ครับ/ค่ะ",
            ],
            "B",
            done,
            total_steps,
        )
        set_quota_project_gate()
        stage = 5
        save_stage(stage)
        done = progress_from_stage(stage)

    if stage <= 5:
        ask_until_correct(
            "Lv.4 Clear Ask",
            "RO-B0T: ประโยคขอความช่วยเหลือแบบไหน 'ตอบง่าย' ที่สุด?",
            [
                "A) รบกวนอาจารย์ช่วยดู 2 จุดนี้ได้ไหมครับ/ค่ะ: (1)... (2)... และถ้าสะดวกขอนัด 10 นาทีช่วง...",
                "B) อาจารย์ช่วยแนะนำหน่อยครับ/ค่ะ ผมไม่รู้จะเริ่มยังไง",
            ],
            "A",
            done,
            total_steps,
        )
        service = connect_calendar_with_scope_reauth(QUOTA_PROJECT_ID)
        if service is None:
            ui_card(
                "⛔ ยังไปด่านถัดไปไม่ได้: Calendar auth/scope ยังไม่พร้อม",
                "err",
            )
            save_stage(5)
            return
        stage = 6
        save_stage(stage)
        done = progress_from_stage(stage)

    if stage <= 6:
        ask_until_correct(
            "Obstacle #2 Spam Trap",
            "RO-B0T: ก่อนส่งจริง ต้องกัน 'ตำนานส่งผิดคน/ลืมแนบไฟล์' ยังไง?",
            [
                "A) Preview อีกรอบ + ตรวจ To/CC + แนบไฟล์ + ชื่อไฟล์สื่อความ + deadline/next step",
                "B) พิมพ์ว่า 'ตามไฟล์แนบ' แล้วค่อยไปแนบทีหลัง",
            ],
            "A",
            done,
            total_steps,
        )
        stage = 7
        save_stage(stage)
        done = progress_from_stage(stage)

    if stage <= 7:
        ask_until_correct(
            "Lv.5 Last Boss",
            "RO-B0T: จะกดส่งได้ ต้องทำขั้นตอนสุดท้ายอะไร?",
            [
                "A) กด Send เลย ความมั่นใจคือพลัง",
                "B) พิมพ์คำสั่งยืนยัน เช่น 'SEND' เพื่อกันมือพลาด",
            ],
            "B",
            done,
            total_steps,
        )
        stage = 8
        save_stage(stage)
        done = progress_from_stage(stage)

    if service is None:
        service = connect_calendar_with_scope_reauth(QUOTA_PROJECT_ID)
        if service is None:
            ui_card("⛔ Calendar API ยังไม่พร้อม จึงจบเกมต่อไม่ได้", "err")
            return
    list_events_today_and_range(service)
    quick_add_and_notify(service)
    ui_hero("You are Winner!!!", f"Quest cleared {done}/{total_steps}")
    clear_stage()


if __name__ == "__main__":
    main()



Your answer (a/b): ิิb



Your answer (a/b): b



Your answer (a/b): a



Your answer (a/b): b


[RUN] gcloud auth application-default print-access-token --quiet


...


Your answer (a/b): a



Your answer (a/b): a



Your answer (a/b): b


ID,Date,Time,Title
#01,2026-02-01,09:30–10:30,Test
#02,2026-02-12,09:00–10:00,Event Subject
#03,2026-02-13,09:00–10:00,Consult/Feedback: MADT7204
#04,2026-02-14,09:00–10:00,Play Game
#05,2026-02-20,09:00–10:00,Play Game
#06,2026-02-20,09:00–10:00,Event Subject
#07,2026-02-22,09:00–12:00,play game


Guardrail: create only 08:00–18:00 (15-min steps)
Tip: พิมพ์ q ช่องไหนก็ได้ เพื่อหยุดแล้วค่อยมานัดต่อทีหลัง

Title: q
